In [ ]:
pip install -q langchain chromadb huggingface_hub tavily-python google-generativeai


In [ ]:
pip install -U langchain-community

In [ ]:
import os
import google.generativeai as genai

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
TAVILY_API_KEY = UserSecretsClient().get_secret("TAVILY_API_KEY")
# print(GOOGLE_API_KEY,TAVILY_API_KEY)
# Set your API keys
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [ ]:
pip install -U langchain langchain-google-genai


In [25]:
import json
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools.tavily_search import TavilySearchResults


In [26]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)

In [27]:
data=[
  {
    "id": "001",
    "event": "US Imposes 104% Tariff on Chinese Imports",
    "date": "2025-04-09",
    "region": "USA",
    "summary": "President Trump's administration has implemented a 104% tariff on Chinese imports, escalating trade tensions between the two nations.",
    "source": "https://www.businessinsider.com/trump-tariffs-are-here-2025-4",
    "category": "trade policy"
  },
  {
    "id": "002",
    "event": "China Retaliates with 84% Tariff on US Goods",
    "date": "2025-04-09",
    "region": "China",
    "summary": "In response to US tariffs, China has announced an 84% tariff on US goods, effective April 10, 2025.",
    "source": "https://www.aljazeera.com/news/liveblog/2025/4/9/trump-tariffs-stocks-dive-as-world-braces-for-duties-to-begin-at-midnight",
    "category": "trade policy"
  },
  {
    "id": "003",
    "event": "India Unlikely to Retaliate Against US Tariffs",
    "date": "2025-04-06",
    "region": "India",
    "summary": "Indian government officials indicate that India does not plan to retaliate against the 26% tariff imposed by the US, citing ongoing trade deal negotiations.",
    "source": "https://www.reuters.com/world/india-unlikely-retaliate-against-trumps-tariffs-deal-talks-progress-sources-say-2025-04-06/",
    "category": "trade policy"
  },
  {
    "id": "004",
    "event": "EU Seeks Zero Tariff from India on Car Imports",
    "date": "2025-04-07",
    "region": "European Union",
    "summary": "Following US tariff hikes, the European Union is seeking zero tariffs from India on car imports to bolster trade relations.",
    "source": "https://www.reuters.com/business/autos-transportation/after-trump-eu-seeks-zero-tariff-india-car-imports-sources-say-2025-04-07/",
    "category": "trade policy"
  },
  {
    "id": "005",
    "event": "Oil Prices Plummet Amid US-China Tariff Escalation",
    "date": "2025-04-09",
    "region": "Global",
    "summary": "Oil prices have dropped nearly 4% as the US imposes 104% tariffs on Chinese goods and China retaliates with increased tariffs on US goods.",
    "source": "https://www.reuters.com/business/energy/oil-slides-nearly-4-us-kicks-off-104-tariffs-china-2025-04-09/",
    "category": "economic impact"
  },
  {
    "id": "006",
    "event": "India's Exports to US Could Decline by $5.76 Billion Due to Tariffs",
    "date": "2025-04-07",
    "region": "India",
    "summary": "Analysts predict that India's exports to the US could face a $5.76 billion decline in 2025 due to recent tariff hikes.",
    "source": "https://timesofindia.indiatimes.com/business/india-business/indias-exports-to-us-could-face-5-76-billion-decline-in-2025-due-to-tariff-hikes/articleshow/120059905.cms",
    "category": "economic impact"
  },
  {
    "id": "007",
    "event": "China Plans High-Level Meeting to Address US Tariffs",
    "date": "2025-04-09",
    "region": "China",
    "summary": "Chinese officials are organizing a high-level meeting to craft support measures in response to the recent US tariff hikes.",
    "source": "https://www.reuters.com/world/china/china-hold-high-level-meeting-response-us-tariffs-say-sources-2025-04-09/",
    "category": "government response"
  },
  {
    "id": "008",
    "event": "India Pursues New Trade Deals Amid Global Tariff Uncertainty",
    "date": "2025-04-08",
    "region": "India",
    "summary": "India is seeking to establish new trade agreements with various countries to mitigate the impact of global tariff escalations.",
    "source": "https://www.theguardian.com/world/2025/apr/08/tariffs-driving-india-to-strike-trade-deals-finance-minister-says-ahead-of-uk-talks",
    "category": "trade negotiations"
  },
  {
    "id": "009",
    "event": "China Vows to 'Fight to the End' Against US Tariff Threats",
    "date": "2025-04-08",
    "region": "China",
    "summary": "Beijing has accused the US of blackmail and has pledged to 'fight to the end' in response to the latest tariff threats.",
    "source": "https://www.theguardian.com/world/2025/apr/08/china-vows-to-fight-to-the-end-against-latest-trump-tariff-threat",
    "category": "diplomatic relations"
  },
  {
    "id": "010",
    "event": "US Stock Markets Volatile Amid Tariff Announcements",
    "date": "2025-04-08",
    "region": "USA",
    "summary": "Major US stock indexes experienced significant volatility as new tariff deadlines approached, reflecting investor uncertainty.",
    "source": "https://www.investors.com/market-trend/stock-market-today/dow-jones-sp500-nasdaq-nvidia-stock-nvda-tsla-pltr-hum/",
    "category": "financial markets"
  }
]


In [28]:
documents = []
for item in data:
    content = f"""
    EVENT: {item['event']}
    DATE: {item['date']}
    REGION: {item['region']}
    SUMMARY: {item['summary']}
    SOURCE: {item['source']}
    CATEGORY: {item['category']}
    """
    documents.append(Document(page_content=content.strip(), metadata={"source": item["source"]}))

# ✂️ Chunk the documents
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [29]:
# 🔍 Create vector store with HuggingFace embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory="./chroma_tariff_db"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [30]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)

# 🔁 Create RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [31]:
# 🌐 Define tools for the agent
search_tool = TavilySearchResults()

tools = [
    Tool(
        name="TariffNewsDB",
        func=rag_chain.run,
        description="Use this to answer questions about known tariff-related geopolitical events."
    ),
    Tool(
        name="WebSearch",
        func=search_tool.run,
        description="Use this to get current or real-time news from the internet."
    )
]

In [36]:
# 🤖 Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🧪 Query the agent
query = "Did the US impose a 104% tariff on Chinese electric vehicles recently?"
response = agent.invoke({"input": query})
# 🔍 Print the full response to check keys
print("\n🧪 Raw Response:\n", response)

# ✅ Then extract the correct field based on available keys
if "result" in response:
    print("\n✅ Final Answer:\n", response["result"])
elif "output" in response:
    print("\n✅ Final Answer:\n", response["output"])
else:
    print("\n❓ Couldn't find expected result. Here's the full response again:")
    print(response)

# print("\n✅ Final Answer:\n", response['result'])

# print("\n📄 Sources:")
# for i, doc in enumerate(response['source_documents']):
#     print(f"\n--- Source {i+1} ---")
#     print(doc.page_content[:1000])  # limit content length
#     if 'source' in doc.metadata:
#         print(f"🔗 Source: {doc.metadata['source']}")



> Entering new AgentExecutor chain...
Thought: I need to check recent news and databases about US tariffs on Chinese electric vehicles to verify this claim.

Action: WebSearch
Action Input: "US 104% tariff Chinese electric vehicles"
Observation: [{'title': "China hits back after Trump's 104% bombshell - Newswire", 'url': 'https://www.newswire.lk/2025/04/09/china-hits-back-after-trumps-104-bombshell/', 'content': 'EDUWIRE\n\nChina hits back after Trump’s 104% bombshell\n\n\n\nChina has announced that it will impose 84% tariffs on certain goods imported from the United States, starting from April 10. The move comes in response to US President Donald Trump’s announcement of 104% tariffs on Chinese electric vehicles.\n\nAccording to China’s Ministry of Finance, the new tariff rate marks an increase from the previously announced 34% on US goods.', 'score': 0.87218446}, {'title': 'Trump tariffs live updates: US has moved forward with 104% China ...', 'url': 'https://finance.yahoo.com/news/

In [22]:
pip install google-generativeai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 11.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.17
    Uninstalling google-ai-generativelanguage-0.6.17:
      Successfully uninstalled google-ai-generativelanguage-0.6.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.2 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.10 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import google.generativeai as genai

# 🛡️ Set your API key
genai.configure(api_key=GOOGLE_API_KEY)

# 📜 List available models
models = genai.list_models()

# 📋 Print all model names and supported generation methods
for model in models:
    print(f"Model Name: {model.name}")
    print(f"  Description: {model.description}")
    print(f"  Supported generation methods: {model.supported_generation_methods}")
    print("-" * 50)


Model Name: models/chat-bison-001
  Description: A legacy text-only model optimized for chat conversations
  Supported generation methods: ['generateMessage', 'countMessageTokens']
--------------------------------------------------
Model Name: models/text-bison-001
  Description: A legacy model that understands text and generates text as an output
  Supported generation methods: ['generateText', 'countTextTokens', 'createTunedTextModel']
--------------------------------------------------
Model Name: models/embedding-gecko-001
  Description: Obtain a distributed representation of a text.
  Supported generation methods: ['embedText', 'countTextTokens']
--------------------------------------------------
Model Name: models/gemini-1.0-pro-vision-latest
  Description: The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.
  Supported generation methods: ['generateCont